In [ ]:
# Copyright 2019 Google LLC
# Copyright 2025 Anonymized Authors

# Licensed under the Apache License, Version 2.0 (the "License"); 
# you may not use this file except in compliance with the License. 
# You may obtain a copy of the License at
# https://www.apache.org/licenses/LICENSE-2.0
"""
This notebook benchmarks GENE and regularized evolution on NAS-Bench-101.
It uses a population size of 50 and mutation rate of 0.72 for RE. 

Requirements: 

-This notebook requires that  tensorflow and numpy be installed within the 
Python environment you are running this script in. 

"""
import json
import matplotlib.pyplot as plt
import os
import numpy as np
import random

import sys; sys.path.append('..')
from utils.nas_utils import load_nasbench,  NASBenchConstants, NASBench, SpecOneHot, run_revolution_search, init_pop
from utils.plotting import plot_all
from IPython.display import clear_output
from typing import Callable, List, Any

nasb = load_nasbench()


In [ ]:
# we define our GENE algorithm: 

def greedy_selection(parents):
    """Selects greedy n parent pairs"""
    p_size = len(parents)
    population = np.asarray([p[0] for p in parents])

    summed_population = []
    for p1 in range(p_size):
        for p2 in range(p_size):
            if p1 < p2: # only upper triangle
                sum = population[p1] + population[p2]
            else:
                sum = 0
            summed_population.append(sum)

    samples = np.argsort(summed_population)[-p_size:]
    row_indices, col_indices = np.meshgrid(np.arange(p_size),np.arange(p_size))
    mapping = np.reshape(np.stack([col_indices,row_indices],axis=-1), [-1,2])
    pairings = mapping[samples]

    paired_parents = []
    for i in range(len(pairings)):
        p1 = parents[pairings[i,0]]
        p2 = parents[pairings[i,1]]
        paired_parents.append((p1,p2))

    return paired_parents


def crossover(nasb, parents):
    """Performs vertical crossover on parent pairs, ensuring valid offspring."""
    
    def cross(p1, p2):
        if p1.original_matrix.shape != p2.original_matrix.shape:
            print("Shape mismatch")
            return p1

        for _ in range(5):
            i = random.randint(1, NASBenchConstants.NUM_VERTICES - 1)
            m = np.copy(p1.original_matrix)
            m[:i, :] = p2.original_matrix[:i, :]
            o = p1.original_ops[:i] + p2.original_ops[i:]
            new_spec = SpecOneHot(m, o)
            if nasb.is_valid(new_spec):
                return new_spec
        return p1

    return [cross(p[0][1], p[1][1]) for p in parents]


# guided mutation for nasbench specific encoding
def guided_mutation(nasb, population, variant):
    psize = len(population)
    
    inputs = [x.to_flat() for x in population]
    mutation_inputs = np.array(inputs)
    summed = np.sum(mutation_inputs, axis=0)

    # Define the probabilities for mutation
    probs_mat_1 = summed[:SpecOneHot.flat_matrix:2] / psize 
    probs_mat_1 /= np.sum(probs_mat_1)  
    probs_mat_0 = 1 - probs_mat_1  
    probs_mat_0 /= np.sum(probs_mat_0)  

    probs_ops_1 = summed[SpecOneHot.flat_matrix:] / psize
    probs_ops_1 /= np.sum(probs_ops_1)  
    probs_ops_0 = 1 - probs_ops_1
    probs_ops_0 /= np.sum(probs_ops_0)  

    if variant==0:
        probs_ops = probs_ops_0
    else:
        probs_ops = probs_ops_1

    children = []
    for i in range(psize):
        while True:
            child_spec = mutation_inputs[i, :].astype(int)

            # Select indices based on the probabilities using np.random.choice (similar to torch.multinomial)
            mat_index_1 = np.random.choice(len(probs_mat_0), 1, p=probs_mat_0)[0]
            mat_index_0 = np.random.choice(len(probs_mat_1), 1, p=probs_mat_1)[0]

            ops_index = np.random.choice(len(probs_ops), 1, p=probs_ops)[0] + SpecOneHot.flat_matrix

            # Modify the child_spec array based on the selected indices
            base = (ops_index - (ops_index % 3))  # Ensure base is aligned to 3 operations
            remove = [base, base + 1, base + 2]
            child_spec[remove] = 0
            child_spec[ops_index] = 1

            # mutate adjacency matrix
            child_spec[mat_index_1*2] = 1
            child_spec[mat_index_1*2+1] = 0

            child_spec[mat_index_0*2+1] = 1
            child_spec[mat_index_0*2] = 0

            spec = SpecOneHot.spec_from_flat(child_spec)
            
            if nasb.is_valid(spec):
                children.append(spec)
                break

    return children


def run_xevolution(
    nasbench: NASBench,
    selection_algorithm: Callable[[List], Any],
    crossover_algorithm: Callable[[NASBench, List], Any],
    mutation_algorithm: Callable[[NASBench, List], Any],
    max_time_budget=5e6,
    p_size=50,    
):
    """Run a single roll-out of evolution to a fixed time budget."""
    population, times, best_valids, best_tests = init_pop(nasbench, max_time_budget, p_size)
    while True:
        selected_candidates = selection_algorithm(population)  
        crossed_parents = crossover_algorithm(nasbench, selected_candidates)
        children = mutation_algorithm(nasbench, crossed_parents)
        population = []
        for child in children: 
            data = nasbench.query(child)
            time_spent, _ = nasbench.get_budget_counters()
            times.append(time_spent)
            population.append((data["validation_accuracy"], child))
            if data["validation_accuracy"] > best_valids[-1]:
                best_valids.append(data["validation_accuracy"])
                best_tests.append(data["test_accuracy"])
            else:
                best_valids.append(best_valids[-1])
                best_tests.append(best_tests[-1])

        if time_spent> max_time_budget:
            break

    return times, best_valids, best_tests


In [ ]:
# comparing GENE-0, GENE-1 and regularized evolution
gm0 = lambda nasb, population : guided_mutation(nasb, population, 0)
gm1 = lambda nasb, population : guided_mutation(nasb, population, 1)

exp1 = {
    "data": 
    {
        "GENE-0" : [[],"Dark Blue"],
        "GENE-1" : [[],"Red Orange"],
        "RE" : [[],"Very Dark Gray"],

    },
    "config": 
    {
        "budget" : int(1e7),
        "limits" : (0.9395, 0.9435),
        "n" : 1000,
        "print_every" : 2,
        "confidence_intervall" : True,
        "pvalue" : 0.05,
        "significant_areas": False,
        "dataset" : "test",
   }
}

budget = exp1["config"]["budget"]
for run in range(exp1["config"]["n"]):
    nasb.reset_budget_counters()
    times, best_valid, best_test = run_xevolution(nasb, greedy_selection, crossover, gm0, budget, 50)
    exp1["data"]["GENE-0"][0].append((times, best_valid, best_test))

    nasb.reset_budget_counters()
    times, best_valid, best_test = run_xevolution(nasb, greedy_selection, crossover, gm1, budget, 50)
    exp1["data"]["GENE-1"][0].append((times, best_valid, best_test))

    nasb.reset_budget_counters()
    times, best_valid, best_test = run_revolution_search(nasb, budget, 50,10,0.72)
    exp1["data"]["RE"][0].append((times, best_valid, best_test))


    if (run % exp1["config"]["print_every"] == 0):
        clear_output(wait=True)
        fig, ax = plt.subplots()
        plot_all(exp1, ax)
        plt.show()
        print('Running repeat %d' % (run + 1))

clear_output(wait=True)
fig, ax = plt.subplots()
means1 , _ =  plot_all(exp1, ax)
plt.savefig('GENE_benchmark_all.png', dpi=600)
plt.show()

In [ ]:
# save results to folder
! this could overwrite existing files

date = "3_17"
f1 = os.path.join("..","generated","models", "GENE", f"GENE-0_1e7_S_1000_{date}.json")
f2 = os.path.join("..","generated","models", "GENE", f"GENE-1_1e7_S_1000_{date}.json")
f3 = os.path.join("..","generated","models", "GENE", f"RE_1e7_S1000_{date}.json")

with open(f1, 'w') as file:
    json.dump(exp1["data"]["GENE-0"][0], file)

with open(f2, 'w') as file:
    json.dump(exp1["data"]["GENE-1"][0], file)

with open(f3, 'w') as file:
    json.dump(exp1["data"]["RE"][0], file)

In [ ]:
# only print specific plots from import

exp1a = {
    "data": 
    {
        # "GENE-0" : [[],"Dark Blue"], # comment which one to plot
        "GENE-1" : [[],"Dark Blue"],
        "RE" : [[],"Very Dark Gray"],

    },
    "config": 
    {
        "budget" : int(1e7),
        "limits" : (0.9395, 0.9435),
        "n" : 1000,
        "print_every" : 2,
        "confidence_intervall" : True,
        "pvalue" : 0.05,
        "significant_areas": False,
        "dataset" : "test",
    }
}


f3 = os.path.join("..","generated","models", "GENE", "new_gm1_1e7.json")
f4 = os.path.join("..","generated","models", "GENE", "new_re_1e7.json")

with open(f3, 'r') as file:
    exp1a["data"]["GENE-1"][0] = json.load(file)

with open(f4, 'r') as file:
    exp1a["data"]["RE"][0] = json.load(file)
    
print("importing done.")
fig, ax = plt.subplots()
means1a , _ =  plot_all(exp1a, ax)
# plt.savefig('GENE-1_1e7_S_1000.png', dpi=600)
plt.show()

In [ ]:
# only print specific plots from previous experiment

exp1b = {
    "data": 
    {
        "GENE-0" : [exp1["data"]["GENE-0"][0],"Dark Blue"],
        # "GENE-1" : [exp1["data"]["GENE-1"][0],"Dark Blue"],
        "RE" : [exp1["data"]["RE"][0],"Very Dark Gray"],

    },
    "config":  exp1["config"]
}

clear_output(wait=True)
fig, ax = plt.subplots()
means1b , stds =  plot_all(exp1b, ax)
# plt.savefig('GENE-0_1e7_S_1000.png', dpi=600)

plt.show()

In [ ]:
# plotting means


steps = len(means1[0]) 
print(steps)
thresholds = [0.938,0.939, 0.940]

gm0 = []
gm1 = []
rr= []
for t in thresholds:
    for i in range(steps):
        if means1[0][i] > t:
            gm0.append(i)
            break
    for i in range(steps):
        if means1[1][i] > t:
            gm1.append(i)
            break
    for i in range(steps):
        if means1[2][i] > t:
            rr.append(i)
            break
print(gm0)
print(rr)

print("thr |  GENE-0 | GENE-1  |RE | speedup of gm over rr ")
for i in range(len(thresholds)):
    if gm1[i] <= rr[i]:
        speed_up = "+" + str(int(100-(gm1[i]/rr[i]*100))) + "%"
    else: 
        speed_up = str(int(100-(gm1[i]/rr[i]*100))) + "%"
    print(f"{thresholds[i]:.6} | {gm1[i]} | {gm1[i]}  |  {rr[i]} |  {speed_up} ")



In [ ]:

formatted_thresholds = [f"{t:.4f}" for t in thresholds[:-1]]
formatted_thresholds.append("max")
speedups = []
for i in range(len(thresholds)):
    if gm1[i] <= rr[i]:
        speedup = "+" + str(int(100 - (gm1[i] / rr[i] * 100))) + "\\%"
    else:
        speedup = str(int(100 - (gm1[i] / rr[i] * 100))) + "\\%"
    speedups.append(speedup)
    
print("\\begin{tabular}{|c|" + "c|" * len(formatted_thresholds) + "}")
print("\\hline")
print("accuracy & " + " & ".join(formatted_thresholds) + " \\\\")
print("\\hline")
print("GENE-0 & " + " & ".join([str(g) for g in gm0]) + " \\\\")
print("\\hline")
print("GENE-1 & " + " & ".join([str(g) for g in gm1]) + " \\\\")
print("\\hline")
print("RE & " + " & ".join([str(r) for r in rr]) + " \\\\")
print("\\hline")
print("speedup & " + " & ".join(speedups) + " \\\\")
print("\\hline")
print("\\end{tabular}")
